<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- [X] Fit a Gensim LDA topic model on Amazon Reviews
- [X] Select appropriate number of topics
- [X] Create some dope visualization of the topics
- [ ] Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [78]:
#Start Here
import numpy as np
import gensim
import os
import re
import sys

# lda is in scikit learn but gensim is way better apparently
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
import pyLDAvis.gensim

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

/home/ygeun/anaconda3/envs/U4W1-NLP/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


In [59]:
def tokenize(text):
    return [token for token in simple_preprocess(text, deacc=True) if token not in STOPWORDS]

In [60]:
df['tk'] = df['reviews.text'].apply(lambda x: tokenize(x))

In [61]:
df.head(2)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,tk
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...","[order, item, bad, quality, missing, backup, s..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...","[bulk, expensive, way, products, like]"


In [62]:
# A Dictionary Representation of all the words in our corpus
id2word = corpora.Dictionary(df.tk)

In [63]:
def print_id2word_stats(id2word):
    print(f'''
    id2word size: {sys.getsizeof(id2word)}
    id2word length: {len(id2word.keys())}''')

In [64]:
print_id2word_stats(id2word)


    id2word size: 56
    id2word length: 9621


In [65]:
# Filter the tippy top words, and the bottom words
id2word.filter_extremes(no_below=10, no_above=.975)

In [66]:
print_id2word_stats(id2word)


    id2word size: 56
    id2word length: 2359


In [67]:
# This is like countVectorize.transform or something
corpus = [id2word.doc2bow(t) for t in df.tk]

In [70]:
corpus[0][:10]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]

NOW WE READY FOR LDA

In [71]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word, # dict not necessary for fitting of model but handy later on for visz
                   random_state=444666,
                   num_topics = 10,
                   passes=5, # it means the # of times I fit those distributions
                   workers=2
                  )

Not entirely sure what it means to select an apropos amount of topics

In [72]:
lda.print_topics()

[(0,
  '0.038*"work" + 0.028*"batteries" + 0.026*"amazon" + 0.017*"model" + 0.016*"cheap" + 0.014*"like" + 0.014*"need" + 0.013*"nephew" + 0.013*"battery" + 0.012*"good"'),
 (1,
  '0.059*"ipad" + 0.034*"good" + 0.029*"works" + 0.024*"features" + 0.022*"quality" + 0.019*"like" + 0.018*"cover" + 0.018*"content" + 0.016*"slow" + 0.016*"price"'),
 (2,
  '0.037*"love" + 0.028*"amazon" + 0.022*"buy" + 0.019*"access" + 0.018*"kindle" + 0.018*"best" + 0.014*"store" + 0.014*"better" + 0.014*"like" + 0.013*"app"'),
 (3,
  '0.038*"kindle" + 0.025*"grandson" + 0.021*"reading" + 0.019*"bought" + 0.017*"friendly" + 0.015*"great" + 0.015*"case" + 0.015*"camera" + 0.014*"user" + 0.012*"book"'),
 (4,
  '0.121*"tablet" + 0.039*"kids" + 0.034*"great" + 0.033*"apps" + 0.027*"use" + 0.025*"games" + 0.023*"love" + 0.019*"amazon" + 0.018*"easy" + 0.016*"play"'),
 (5,
  '0.014*"wifi" + 0.013*"battery" + 0.012*"use" + 0.011*"sound" + 0.010*"keeps" + 0.010*"resolution" + 0.009*"good" + 0.009*"warranty" + 0.009*

In [73]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [75]:
topics = [' '.join(t[0:5]) for t in words]

In [76]:
topics

['work batteries amazon model cheap',
 'ipad good works features quality',
 'love amazon buy access kindle',
 'kindle grandson reading bought friendly',
 'tablet kids great apps use',
 'wifi battery use sound keeps',
 'loves bought old year gift',
 'batteries long brand amazon battery',
 'kindle screen easy use size',
 'great price good product value']

In [79]:
pyLDAvis.enable_notebook()

In [80]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/home/ygeun/anaconda3/envs/U4W1-NLP/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.197512 -0.082110       1        1  17.404715
4     -0.141048  0.050907       2        1  17.284943
8     -0.133979 -0.056728       3        1  11.799998
9      0.230217  0.241429       4        1   9.253581
6     -0.188343  0.144159       5        1   9.013471
5      0.002075 -0.163735       6        1   8.243997
2     -0.053898 -0.086051       7        1   7.566769
3     -0.174229  0.022277       8        1   7.156018
0      0.152468 -0.124573       9        1   6.759335
1      0.109225  0.054425      10        1   5.517170, topic_info=     Category         Freq       Term        Total  loglift  logprob
1831  Default  6420.000000     tablet  6420.000000  30.0000  30.0000
20    Default  8978.000000      great  8978.000000  29.0000  29.0000
17    Default  6944.000000  batteries  6944.000000  28.0000  28.0000
16    Default  5077.000000      price  5077.000000  27.0000  27.0000
33    Default  5652.000000       good  5652.000000  26.0000  26.0000
1496  Default  2267.000000      loves  2267.000000  25.0000  25.0000
1627  Default  3004.000000     kindle  3004.000000  24.0000  24.0000
1370  Default  1586.000000     screen  1586.000000  23.0000  23.0000
481   Default  2198.000000        old  2198.000000  22.0000  22.0000
1286  Default  1704.000000       apps  1704.000000  21.0000  21.0000
1512  Default   982.000000       ipad   982.000000  20.0000  20.0000
158   Default  2174.000000    product  2174.000000  19.0000  19.0000
27    Default  3209.000000     bought  3209.000000  18.0000  18.0000
8     Default  2194.000000       work  2194.000000  17.0000  17.0000
101   Default  1820.000000       year  1820.000000  16.0000  16.0000
22    Default  2605.000000       long  2605.000000  15.0000  15.0000
785   Default  1109.000000       gift  1109.000000  14.0000  14.0000
679   Default  2900.000000       easy  2900.000000  13.0000  13.0000
19    Default  1302.000000      brand  1302.000000  12.0000  12.0000
594   Default  1668.000000      games  1668.000000  11.0000  11.0000
185   Default  2379.000000       kids  2379.000000  10.0000  10.0000
360   Default   962.000000      value   962.000000   9.0000   9.0000
92    Default  1548.000000      works  1548.000000   8.0000   8.0000
86    Default  3429.000000       love  3429.000000   7.0000   7.0000
6     Default  1307.000000    quality  1307.000000   6.0000   6.0000
1633  Default  1490.000000      books  1490.000000   5.0000   5.0000
1086  Default   969.000000   daughter   969.000000   4.0000   4.0000
30    Default   874.000000  christmas   874.000000   3.0000   3.0000
23    Default  3845.000000     amazon  3845.000000   2.0000   2.0000
90    Default  4373.000000        use  4373.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
962   Topic10    40.471363       miss    48.904964   2.7080  -6.0107
1841  Topic10   402.651306   features   618.613770   2.4679  -3.7132
859   Topic10   146.008179      apple   205.205963   2.5570  -4.7276
1027  Topic10   102.594910  bluetooth   142.097519   2.5716  -5.0805
1780  Topic10    35.370636      photo    42.795059   2.7068  -6.1454
1333  Topic10    41.221027       baby    52.353474   2.6582  -5.9923
957   Topic10    51.558784      asked    69.650772   2.5965  -5.7685
1617  Topic10    56.692310      voice    78.467934   2.5723  -5.6736
1034  Topic10    88.849823    parents   139.859772   2.4436  -5.2243
1822  Topic10   128.696198    speaker   226.185272   2.3334  -4.8538
92    Topic10   478.446472      works  1548.831299   1.7226  -3.5407
1194  Topic10   199.356842      music   482.571625   2.0133  -4.4162
1391  Topic10    59.241886  listening    88.411041   2.4969  -5.6296
6     Topic10   357.424957    quality  1307.990845   1.6000  -3.8323
247   Topic10   131.718079    amazing   350.469452   1.9187  -4.8306
305   Topic10   133.167648        low   400.955627   1.7951  -4.81

- I don't really have a clue as to what is happening with this assignment, so providing a cogent analysis would be a little moot.

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling